# DATA SENTIMEN

In [130]:
import pandas as pd
tweets = pd.read_excel('Data/Data Sentimen/Labeled Data/labelled-validation.xlsx')

In [ ]:
tweets.head()

In [ ]:
tweets.info()

In [ ]:
tweets.tail()

In [ ]:
tweets['date'] = pd.to_datetime(tweets['date'])

sentiment_counts = tweets.groupby([tweets['date'].dt.date, 'labels']).size().unstack(fill_value=0)

sentiment_counts.reset_index(inplace=True)

sentiment_counts.columns.name = None 
sentiment_counts.columns = ['date', 'negatif', 'netral', 'positif']

sentiment_counts['total'] = sentiment_counts['negatif'] + sentiment_counts['netral'] + sentiment_counts['positif']

sentiment_counts['date'] = pd.to_datetime(sentiment_counts['date']).dt.strftime('%Y-%m-%d')

In [ ]:
sentiment_counts.head(50)

In [ ]:
sentiment_counts[sentiment_counts['negatif'] == sentiment_counts['positif']]

In [ ]:
# Plot
import matplotlib.pyplot as plt

dates = sentiment_counts.index
negatif = sentiment_counts['negatif']
netral = sentiment_counts['netral']
positif = sentiment_counts['positif']

plt.figure(figsize=(12, 6))
plt.plot(dates, negatif, label='Negatif', color='red')
plt.plot(dates, netral, label='Netral', color='blue')
plt.plot(dates, positif, label='Positif', color='green')

plt.xlabel('Tanggal')
plt.ylabel('Jumlah Sentimen')
plt.title('Perbandingan Jumlah Sentimen')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [138]:
def determine_sentiment(row):
    if row['negatif'] > 0 and row['positif'] == 0 and row['netral'] == 0:
        return 'negatif'
    elif row['positif'] > 0 and row['negatif'] == 0 and row['netral'] == 0:
        return 'positif'
    elif row['netral'] > 0:
        if row['negatif'] > 0 and row['positif'] > 0:
            if row['negatif'] > row['positif']:
                return 'negatif'
            elif row['positif'] > row['negatif']:
                return 'positif'
            else:
                return 'netral'
        elif row['negatif'] > 0:
            return 'negatif'
        elif row['positif'] > 0:
            return 'positif'
        else:
            return 'netral'
    elif row['negatif'] > 0 and row['positif'] == 0:
        return 'negatif'
    elif row['positif'] > 0 and row['negatif'] == 0:
        return 'positif'
    else:
        return 'netral'

# Apply the function to each row
sentiment_counts['Sentimen'] = sentiment_counts.apply(determine_sentiment, axis=1)

In [ ]:
sentiment_counts.head(50)

In [ ]:
sentiment_counts['Sentimen'].value_counts()

In [ ]:
dataset_sentimen = sentiment_counts[['date', 'Sentimen']]
dataset_sentimen.tail()

In [142]:
dataset_sentimen.to_excel('Data/Data Sentimen/dataset_sentimen_validation.xlsx', index=False)

# GOOGLE TRENDS

In [ ]:
gtrend = pd.read_csv('Data/Data Google Trends/Validation.csv')
gtrend.tail()

In [ ]:
sentimen = pd.read_excel('Data/Data Sentimen/dataset_sentimen_validation.xlsx')
sentimen.tail()

In [ ]:
len(sentiment_counts)

In [ ]:
len(gtrend)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(gtrend['volume'])
plt.show

# GABUNGKAN SENTIMEN & GOOGLE TRENDS

In [ ]:
sentimen.dtypes

In [ ]:
gtrend.dtypes

In [154]:
sentimen['date'] = pd.to_datetime(sentimen['date'])
gtrend['hari'] = pd.to_datetime(gtrend['hari'])

In [ ]:
sentimen.dtypes

In [ ]:
gtrend.dtypes

In [157]:
sentimen_gtrend = pd.concat([sentimen, gtrend], axis=1)

In [ ]:
sentimen_gtrend

In [159]:
sentimen_gtrend = sentimen_gtrend.drop(columns='hari')

In [160]:
sentimen_gtrend = sentimen_gtrend.rename(columns={'volume': 'Index'})

In [ ]:
sentimen_gtrend

In [ ]:
sentimen_gtrend.dtypes

In [ ]:
sentimen_gtrend.info()

# PRICE DATA

In [ ]:
price = pd.read_csv('../Data/Data Historis Bitcoin/Validation.csv')
price.head()

In [ ]:
price.dtypes

In [166]:
price['Date'] = pd.to_datetime(price['Date'])

In [ ]:
price.dtypes

In [ ]:
price.tail()

In [169]:
all_data = pd.concat([price, sentimen_gtrend], axis=1)

In [ ]:
all_data.tail()

In [171]:
all_data = all_data.drop(columns = 'date')

In [ ]:
all_data.tail()

In [173]:
columns = list(all_data.columns)
columns.remove('Close')  
columns.append('Close')  
all_data = all_data[columns] 

In [ ]:
all_data

In [175]:
all_data = all_data.rename(columns={'Adj Close': 'Adj_Close'})

In [ ]:
all_data

In [177]:
all_data.to_excel('Data/all_data_validation.xlsx', index=False)